(!) **Note**: We want to compare polarised flux to total intensity flux

    i.e. Fobject,pol/Fstar,pol and Fobject,ti/Fobject,ti
    
  

In [2]:
from import_functions_generic import * 
from import_functions_plotting import * 
#from image_manipulation import * 

In [140]:
L = time.localtime()
date = "{}-{}-{}".format(L[0],L[1],L[2],L[3],L[4],L[5])

saving_dir = 'figs/fig_v{}/'.format(date)
os.makedirs(saving_dir,exist_ok=True)

# 1. From (forward-)modelling results 


**Forward modelling results:**

Inputs: 
- IRDAP image (ADU) and associated PSF normalized to 1
- PCA image (ADU) and associated PSF normalized to 1

So in principle I would only need to correct the fitted scaling values by DIT and TRANSMISSION.

Note: the PSF is normalized in the FM code as follows: sum over all pixels with a PSF of shape (25,25) in polar (value = 723584) and (25,25) in total intensity (value = 9028321), so consistent!

DO THAT TOMORROW's MORNING



By modelling the disk we want to minimize the residuals expressed as follows

$$    \text{residuals} \;=\; \frac{\text{reduced_obs_image} \times \text{mask} -\text{synthetic_model_disk}*\text{PSF}}{\text{noise}}\,. $$

with

- the sum of the PSF values are normalized to 1

- the mask is a binary mask with 0 or 1 values

- '$*$' stands for convolution

The model of the disk is computed with the $\texttt{VIP}$ function  $\texttt{ScatteredLightDisk(...)}$ and then multiply by a $scaling$ parameter that is fitted with the $\texttt{AMOEBA}$ exploration of the parameter space, i.e.

$$\text{synthetic_model_disk} = \texttt{ScatteredLightDisk(...)} \times scaling$$

**Notes**
as the PSF is normalized to 1, the total flux of the disk or the convolved disk is the same.



# Polarimetry

**Goal:** From Qphi image, we have values in ADU We want to convert this ADU value in a flux value.


$$\text{QPHI (ADU)} \; \rightarrow\; \text{QPHI (Jy)}\; =\; \frac{\text{QPHI (ADU)}}{\text{STAR (ADU)}} \times \frac{\text{DIT}_\star}{\text{DIT}_\text{QPHI}} \times \frac{\text{TRANSMISSION}_\star}{\text{TRANSMISSION}_\text{QPHI}}  \times \frac{1}{\text{PLATESCALE (arcsec/pixel)}^2} \times \text{STAR (Jy)} $$  

In [31]:
# Corresponding function

def ADU2FLUX(flux_adu_obj, flux_adu_star, flux_ref_star, dit_obj, dit_star, transmi_obj, transmi_star, platescale):
    x = flux_adu_star / (dit_star * transmi_star) 
    #print('Star flux = %.3e ADU/s' % x )
    return flux_adu_obj/ flux_adu_star * dit_star/dit_obj * transmi_star/transmi_obj * 1/platescale**2 * flux_ref_star 

Values of DIT_STAR, DIT_OBJECT, TRANSMISSION_STAR, TRANSMISSION_OBJECT are taken from the header of the cubes.

In [3]:
## Params: obs
DIT_STAR, DIT_OBJECT = 0.837464, 64 # s taken from the header of the images
TRANSMISSION_STAR, TRANSMISSION_OBJECT = 0.12591487665785145, 1 # corresponds to ND1.0 and OPEN; taken from the header of the images
PLATESCALE =  12.25e-3 # arcsec/pix 
FLUX_REF_STAR = 0.975 # Jy from Simbad in the filter Johnson H

#### Results from fitting only polarimetry obs

In [4]:
## Params: flux
scaling = 1393405 # fitted parameter; not used
flux_adu_obj = 18647 # total flux of the disk after multplying by the scaling parameter
#disk_bef_scaling = flux_adu_obj/scaling

#1514654 # taken equal to the fitter flux scaling parameter in the modelling of debris disk 
flux_adu_star = 723584 # within a box region of size = 25 pixels, left+right images

print('(results *not* corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))


print('\n(results corrected from DITs and transmission)')
print('The flux of the star is {:.2e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

(results *not* corrected from DITs and transmission)
The flux of the star is 7.2e+05 ADU.
The flux of the disk is 18647.00 ADU.

(results corrected from DITs and transmission)
The flux of the star is 6.86e+06 ADU/s.
The flux of the disk is 291.36 ADU/s


In [103]:
flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.2e} Jy/arcsec^2.'.format(flux_obj_final_unit))

Star flux = 6.862e+06 ADU/s
The flux of the disk is 2.76e-01 Jy/arcsec^2.


#### Results from joint-fitting polarimetry+total intensity in both BBH bands

##### for the total intensity-part: used 5 modes, 1 channel

In [104]:
## Params: flux
scaling = 1514654 # fitted parameter; not used
flux_adu_obj = 18794 # total flux of the disk after multplying by the scaling parameter
#disk_bef_scaling = flux_adu_obj/scaling

#1514654 # taken equal to the fitter flux scaling parameter in the modelling of debris disk 
flux_adu_star = 723584 # within a box region of size = 25 pixels, left+right images

print('(results *not* corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))


print('\n(results corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

(results *not* corrected from DITs and transmission)
The flux of the star is 7.2e+05 ADU.
The flux of the disk is 18794.00 ADU.

(results corrected from DITs and transmission)
The flux of the star is 6.9e+06 ADU/s.
The flux of the disk is 293.66 ADU/s


In [105]:
flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.2e} Jy/arcsec^2.'.format(flux_obj_final_unit))

Star flux = 6.862e+06 ADU/s
The flux of the disk is 2.78e-01 Jy/arcsec^2.


##### for the total intensity-part: used 5 modes, both channels

In [5]:
## Params: flux
scaling = 1593331 # fitted parameter; not used
flux_adu_obj = 18838 # total flux of the disk after multplying by the scaling parameter
#disk_bef_scaling = flux_adu_obj/scaling

#1514654 # taken equal to the fitter flux scaling parameter in the modelling of debris disk 
flux_adu_star = 723584 # within a box region of size = 25 pixels, left+right images

print('(results *not* corrected from DITs and transmission)')
print('The flux of the star is {:.2e} ADU.'.format(flux_adu_star))
print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))


print('\n(results corrected from DITs and transmission)')
print('The flux of the star is {:.2e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

(results *not* corrected from DITs and transmission)
The flux of the star is 7.24e+05 ADU.
The flux of the disk is 18838.00 ADU.

(results corrected from DITs and transmission)
The flux of the star is 6.86e+06 ADU/s.
The flux of the disk is 294.34 ADU/s


In [6]:
flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.2e} Jy/arcsec^2.'.format(flux_obj_final_unit))

Star flux = 6.862e+06 ADU/s
The flux of the disk is 2.79e-01 Jy/arcsec^2.


##### for the total intensity-part: used 10 modes, 1 channel

In [108]:
## Params: flux
scaling = 1226197 # fitted parameter; not used
flux_adu_obj = 18453 # total flux of the disk after multplying by the scaling parameter
#disk_bef_scaling = flux_adu_obj/scaling

#1514654 # taken equal to the fitter flux scaling parameter in the modelling of debris disk 
flux_adu_star = 723584 # within a box region of size = 25 pixels, left+right images

print('(results *not* corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))


print('\n(results corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

(results *not* corrected from DITs and transmission)
The flux of the star is 7.2e+05 ADU.
The flux of the disk is 18453.00 ADU.

(results corrected from DITs and transmission)
The flux of the star is 6.9e+06 ADU/s.
The flux of the disk is 288.33 ADU/s


In [110]:
flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.2e} Jy/arcsec^2.'.format(flux_obj_final_unit))

Star flux = 6.862e+06 ADU/s
The flux of the disk is 2.73e-01 Jy/arcsec^2.


##### for the total intensity-part: used 10 modes, both channels

In [7]:
## Params: flux
scaling = 1421607 # fitted parameter; not used
flux_adu_obj = 18749 # total flux of the disk after multplying by the scaling parameter
#disk_bef_scaling = flux_adu_obj/scaling

#1514654 # taken equal to the fitter flux scaling parameter in the modelling of debris disk 
flux_adu_star = 723584 # within a box region of size = 25 pixels, left+right images

print('(results *not* corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))


print('\n(results corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

(results *not* corrected from DITs and transmission)
The flux of the star is 7.2e+05 ADU.
The flux of the disk is 18749.00 ADU.

(results corrected from DITs and transmission)
The flux of the star is 6.9e+06 ADU/s.
The flux of the disk is 292.95 ADU/s


In [9]:
flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.2e} Jy/arcsec^2.'.format(flux_obj_final_unit))

Star flux = 6.862e+06 ADU/s
The flux of the disk is 2.77e-01 Jy/arcsec^2.


#### Results from joint-fitting polarimetry+total intensity in both BBH and H23 bands 

##### for the total intensity-part: used 5 modes, 1 channel

##### for the total intensity-part: used 5 modes, both channels

##### for the total intensity-part: used 10 modes, 1 channel

##### for the total intensity-part: used 10 modes, both channels

In [ ]:
## Params: flux
#scaling = 1421607 # fitted parameter; not used
flux_adu_obj = 18749 # total flux of the disk after multplying by the scaling parameter
#disk_bef_scaling = flux_adu_obj/scaling

#1514654 # taken equal to the fitter flux scaling parameter in the modelling of debris disk 
flux_adu_star = 723584 # within a box region of size = 25 pixels, left+right images

print('(results *not* corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))


print('\n(results corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

In [ ]:
flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.2e} Jy/arcsec^2.'.format(flux_obj_final_unit))

# Total intensity

For the star flux, we have two PSF acquired before and after the observation and for the left and right part of the detector.
We take the mean from before and after the observation and consider either only the left or right cubes, or the sum of them, depending if we work on one or the channels of the SCIENCE cube.

Values of DIT_STAR, DIT_OBJECT, TRANSMISSION_STAR, TRANSMISSION_OBJECT are taken from the header of the cubes.

In [65]:
# Params
DIT_STAR, DIT_OBJECT = 8, 32 # s
TRANSMISSION_STAR, TRANSMISSION_OBJECT = 0.014581900, 1 # ND2.0 and OPEN are used. Corresponding value for ND2.0
# taken from the ESO website https://www.eso.org/sci/facilities/paranal/instruments/sphere/inst/filters.html
PLATESCALE =  12.25e-3 # arcsec/pix 
FLUX_REF_STAR = 0.975  # Jy from Simbad in the filter Johnson H

#### Results from joint-fitting polarimetry+total intensity in both BBH bands

##### for the total intensity-part: used 5 modes, 1 channel

In [66]:
## Params: flux
scaling = 35124 # fitted parameter; not used
flux_adu_obj = 4924 # total flux of the disk after multplying by the scaling parameter
#disk_bef_scaling = flux_adu_obj/scaling

#1514654 # taken equal to the fitter flux scaling parameter in the modelling of debris disk 
flux_adu_star = 9028321/2 # within a box region of size = 25 pixels, left images

print('(results *not* corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))


print('\n(results corrected from DITs and transmission)')
print('The flux of the star is {:.2e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

(results *not* corrected from DITs and transmission)
The flux of the star is 4.5e+06 ADU.
The flux of the disk is 4924.00 ADU.

(results corrected from DITs and transmission)
The flux of the star is 3.87e+07 ADU/s.
The flux of the disk is 153.88 ADU/s


In [67]:
flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.2e} Jy/arcsec^2.'.format(flux_obj_final_unit))

The flux of the disk is 2.58e-02 Jy/arcsec^2.


#### Results from joint-fitting polarimetry+total intensity in both BBH bands

##### for the total intensity-part: used 5 modes, both channels

In [68]:
## Params: flux
scaling = 73490 # fitted parameter; not used
flux_adu_obj = 10645 # total flux of the disk after multplying by the scaling parameter
#disk_bef_scaling = flux_adu_obj/scaling

#1514654 # taken equal to the fitter flux scaling parameter in the modelling of debris disk 
flux_adu_star = 18083288/2 # within a box region of size = 25 pixels, left images

print('(results *not* corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))


print('\n(results corrected from DITs and transmission)')
print('The flux of the star is {:.2e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

(results *not* corrected from DITs and transmission)
The flux of the star is 9.0e+06 ADU.
The flux of the disk is 10645.00 ADU.

(results corrected from DITs and transmission)
The flux of the star is 7.75e+07 ADU/s.
The flux of the disk is 332.66 ADU/s


In [69]:
flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.2e} Jy/arcsec^2.'.format(flux_obj_final_unit))

The flux of the disk is 2.79e-02 Jy/arcsec^2.


#### Results from joint-fitting polarimetry+total intensity in both BBH bands

##### for the total intensity-part: used 10 modes, 1 channel

In [70]:
## Params: flux
scaling = 26412 # fitted parameter; not used
flux_adu_obj = 3545 # total flux of the disk after multplying by the scaling parameter
#disk_bef_scaling = flux_adu_obj/scaling

#1514654 # taken equal to the fitter flux scaling parameter in the modelling of debris disk 
flux_adu_star = 9028321/2 # within a box region of size = 25 pixels, left images

print('(results *not* corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))


print('\n(results corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

(results *not* corrected from DITs and transmission)
The flux of the star is 4.5e+06 ADU.
The flux of the disk is 3545.00 ADU.

(results corrected from DITs and transmission)
The flux of the star is 3.9e+07 ADU/s.
The flux of the disk is 110.78 ADU/s


In [71]:
flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.1e} Jy/arcsec^2.'.format(flux_obj_final_unit))

The flux of the disk is 1.9e-02 Jy/arcsec^2.


#### Results from joint-fitting polarimetry+total intensity in both BBH bands

##### for the total intensity-part: used 10 modes, both channels

In [75]:
## Params: flux
scaling = 56923 # fitted parameter; not used
flux_adu_obj = 7638 # total flux of the disk after multplying by the scaling parameter
#disk_bef_scaling = flux_adu_obj/scaling

#1514654 # taken equal to the fitter flux scaling parameter in the modelling of debris disk 
flux_adu_star = 18083288/2 # within a box region of size = 25 pixels, left images

print('(results *not* corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))


print('\n(results corrected from DITs and transmission)')
print('The flux of the star is {:.2e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

(results *not* corrected from DITs and transmission)
The flux of the star is 9.0e+06 ADU.
The flux of the disk is 7638.00 ADU.

(results corrected from DITs and transmission)
The flux of the star is 7.75e+07 ADU/s.
The flux of the disk is 238.69 ADU/s


In [73]:
flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.2e} Jy/arcsec^2.'.format(flux_obj_final_unit))

The flux of the disk is 2.00e-02 Jy/arcsec^2.


### Results from independent FM 

In [ ]:
DIT_OBJECT_S = [32, 4.001856, 4.001856]
DIT_STAR_S   = [32, 4.001856, 4.001856]

TRANSMISSION_STAR_S = [0.014581900, 0.014581900, 0.014581900]

FLUX_ADU_OBJ  = [8285, 1465, 1715]
FLUX_ADU_STAR = [8245767/2, 1647963/2, 844699]
INFO = ['2019-07-09', '2016-04-05', '2016-06-03']

In [ ]:
DIT_OBJECT_S = [32, 4.001856, 4.001856]
DIT_STAR_S   = [32, 4.001856, 4.001856]

TRANSMISSION_STAR_S = [0.014581900, 0.014581900, 0.014581900]

FLUX_ADU_OBJ  = [3798, 2158, 1898]
FLUX_ADU_STAR = [16512481/2, 3300536/2, 925421]
INFO = ['2019-07-09', '2016-04-05', '2016-06-03']

In [ ]:
DIT_OBJECT_S = [32, 4.001856, 4.001856]
DIT_STAR_S   = [32, 4.001856, 4.001856]

TRANSMISSION_STAR_S = [0.014581900, 0.014581900, 0.014581900]

FLUX_ADU_OBJ  = [3084, 1494, 1603]
FLUX_ADU_STAR = [8245767/2, 1647963/2, 844699]
INFO = ['2019-07-09', '2016-04-05', '2016-06-03']

In [ ]:
DIT_OBJECT_S = [32, 4.001856, 4.001856]
DIT_STAR_S   = [32, 4.001856, 4.001856]

TRANSMISSION_STAR_S = [0.014581900, 0.014581900, 0.014581900]

FLUX_ADU_OBJ  = [9108, 2262, 1339]
FLUX_ADU_STAR = [16512481/2, 3300536/2, 925421]
INFO = ['2019-07-09', '2016-04-05', '2016-06-03']

In [ ]:
## Params: flux

for i in range(len(FLUX_ADU_OBJ)):
    TRANSMISSION_STAR = TRANSMISSION_STAR_S[i]
    DIT_OBJECT, DIT_STAR = DIT_OBJECT_S[i], DIT_STAR_S[i]
    
    flux_adu_obj = FLUX_ADU_OBJ[i] # total flux of the disk after multplying by the scaling parameter
    flux_adu_star = FLUX_ADU_STAR[i] # within a box region of size = 25 pixels, left+right images
    
    print('\n== INFO epoch', INFO[i], '==')
    #print('(results *not* corrected from DITs and transmission)')
    #print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
    #print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))

    print('(results corrected from DITs and transmission)')
    print('The flux of the star is {:.4e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
    print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

    flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
            FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux of the disk is {:.4e} Jy/arcsec^2.'.format(flux_obj_final_unit))

# Polarimetry & Total intensity -- concatenated results

### Results from joint-fitting polarimetry+total intensity in both BBH and H23 bands 

##### for the total intensity-part: used 5 modes, 1 channel

In [63]:
DIT_OBJECT_S = [32, 4.001856, 4.001856, 64]
DIT_STAR_S   = [32, 4.001856, 4.001856, 0.837464]

TRANSMISSION_STAR_S = [0.014581900, 0.014581900, 0.014581900, 0.12591487665785145]

FLUX_ADU_OBJ  = [5043, 907.7, 644.4, 18831]
FLUX_ADU_STAR = [9028321/2, 1768472/2, 925421, 723584]
INFO = ['2019-07-09', '2016-04-05', '2016-06-03', '2018-06-01']

In [39]:
## Params: flux
for i in range(len(FLUX_ADU_OBJ)):
    TRANSMISSION_STAR = TRANSMISSION_STAR_S[i]
    DIT_OBJECT, DIT_STAR = DIT_OBJECT_S[i], DIT_STAR_S[i]
    
    flux_adu_obj = FLUX_ADU_OBJ[i] # total flux of the disk after multplying by the scaling parameter
    flux_adu_star = FLUX_ADU_STAR[i] # within a box region of size = 25 pixels, left+right images
    
    print('\n== INFO epoch', INFO[i], '==')
    #print('(results *not* corrected from DITs and transmission)')
    #print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
    #print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))

    print('(results corrected from DITs and transmission)')
    print('The flux of the star is {:.1e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
    print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

    flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
            FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux of the disk is {:.2e} Jy/arcsec^2.'.format(flux_obj_final_unit))


== INFO epoch 2019-07-09 ==
(results corrected from DITs and transmission)
The flux of the star is 1.9e+07 ADU/s.
The flux of the disk is 157.59 ADU/s
The flux of the disk is 5.29e-02 Jy/arcsec^2.

== INFO epoch 2016-04-05 ==
(results corrected from DITs and transmission)
The flux of the star is 3.0e+07 ADU/s.
The flux of the disk is 226.82 ADU/s
The flux of the disk is 4.86e-02 Jy/arcsec^2.

== INFO epoch 2016-06-03 ==
(results corrected from DITs and transmission)
The flux of the star is 1.6e+07 ADU/s.
The flux of the disk is 161.03 ADU/s
The flux of the disk is 6.60e-02 Jy/arcsec^2.

== INFO epoch 2018-06-01 ==
(results corrected from DITs and transmission)
The flux of the star is 6.9e+06 ADU/s.
The flux of the disk is 294.23 ADU/s
The flux of the disk is 2.79e-01 Jy/arcsec^2.


##### for the total intensity-part: used 5 modes, both channels

In [61]:
DIT_OBJECT_S = [32, 4.001856, 4.001856, 64]
DIT_STAR_S   = [32, 4.001856, 4.001856, 0.837464]
TRANSMISSION_STAR_S = [0.014581900, 0.014581900, 0.014581900, 0.12591487665785145]

FLUX_ADU_OBJ  = [8806, 1442, 1533, 18073]
FLUX_ADU_STAR = [18083288/2, 3557066/2, 1836827, 723584]
INFO = ['2019-07-09', '2016-04-05', '2016-06-03', '2018-06-01']

In [62]:
## Params: flux

for i in range(len(FLUX_ADU_OBJ)):
    TRANSMISSION_STAR = TRANSMISSION_STAR_S[i]
    DIT_OBJECT, DIT_STAR = DIT_OBJECT_S[i], DIT_STAR_S[i]
    
    flux_adu_obj = FLUX_ADU_OBJ[i] # total flux of the disk after multplying by the scaling parameter
    flux_adu_star = FLUX_ADU_STAR[i] # within a box region of size = 25 pixels, left+right images
    
    print('\n== INFO epoch', INFO[i], '==')
    #print('(results *not* corrected from DITs and transmission)')
    #print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
    #print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))


    print('(results corrected from DITs and transmission)')
    print('The flux of the star is {:.1e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
    print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

    flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
            FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux of the disk is {:.2e} Jy/arcsec^2.'.format(flux_obj_final_unit))


== INFO epoch 2019-07-09 ==
(results corrected from DITs and transmission)
The flux of the star is 1.9e+07 ADU/s.
The flux of the disk is 275.19 ADU/s
The flux of the disk is 9.23e-02 Jy/arcsec^2.

== INFO epoch 2016-04-05 ==
(results corrected from DITs and transmission)
The flux of the star is 3.0e+07 ADU/s.
The flux of the disk is 360.33 ADU/s
The flux of the disk is 7.68e-02 Jy/arcsec^2.

== INFO epoch 2016-06-03 ==
(results corrected from DITs and transmission)
The flux of the star is 3.1e+07 ADU/s.
The flux of the disk is 383.07 ADU/s
The flux of the disk is 7.91e-02 Jy/arcsec^2.

== INFO epoch 2018-06-01 ==
(results corrected from DITs and transmission)
The flux of the star is 6.9e+06 ADU/s.
The flux of the disk is 282.39 ADU/s
The flux of the disk is 2.67e-01 Jy/arcsec^2.


##### for the total intensity-part: used 10 modes, 1 channel

In [59]:
DIT_OBJECT_S = [32, 4.001856, 4.001856, 64]
DIT_STAR_S   = [32, 4.001856, 4.001856, 0.837464]
TRANSMISSION_STAR_S = [0.014581900, 0.014581900, 0.014581900, 0.12591487665785145]

FLUX_ADU_OBJ  = [3290, 709.3, 711.1, 18162]
FLUX_ADU_STAR = [9028321/2, 1768472/2, 925421, 723584]
INFO = ['2019-07-09', '2016-04-05', '2016-06-03', '2018-06-01']

In [60]:
## Params: flux

for i in range(len(FLUX_ADU_OBJ)):
    TRANSMISSION_STAR = TRANSMISSION_STAR_S[i]
    DIT_OBJECT, DIT_STAR = DIT_OBJECT_S[i], DIT_STAR_S[i]
    
    flux_adu_obj = FLUX_ADU_OBJ[i] # total flux of the disk after multplying by the scaling parameter
    flux_adu_star = FLUX_ADU_STAR[i] # within a box region of size = 25 pixels, left+right images
    
    print('\n== INFO epoch', INFO[i], '==')
    #print('(results *not* corrected from DITs and transmission)')
    #print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
    #print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))


    print('(results corrected from DITs and transmission)')
    print('The flux of the star is {:.1e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
    print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

    flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
            FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux of the disk is {:.2e} Jy/arcsec^2.'.format(flux_obj_final_unit))


== INFO epoch 2019-07-09 ==
(results corrected from DITs and transmission)
The flux of the star is 9.7e+06 ADU/s.
The flux of the disk is 102.81 ADU/s
The flux of the disk is 6.91e-02 Jy/arcsec^2.

== INFO epoch 2016-04-05 ==
(results corrected from DITs and transmission)
The flux of the star is 1.5e+07 ADU/s.
The flux of the disk is 177.24 ADU/s
The flux of the disk is 7.60e-02 Jy/arcsec^2.

== INFO epoch 2016-06-03 ==
(results corrected from DITs and transmission)
The flux of the star is 1.6e+07 ADU/s.
The flux of the disk is 177.69 ADU/s
The flux of the disk is 7.28e-02 Jy/arcsec^2.

== INFO epoch 2018-06-01 ==
(results corrected from DITs and transmission)
The flux of the star is 6.9e+06 ADU/s.
The flux of the disk is 283.78 ADU/s
The flux of the disk is 2.69e-01 Jy/arcsec^2.


##### for the total intensity-part: used 10 modes, both channels

In [57]:
DIT_OBJECT_S = [32, 4.001856, 4.001856, 64]
DIT_STAR_S   = [32, 4.001856, 4.001856, 0.837464]
TRANSMISSION_STAR_S = [0.014581900, 0.014581900, 0.014581900, 0.12591487665785145]

FLUX_ADU_OBJ  = [6709, 1365, 1113, 18173]
FLUX_ADU_STAR = [18083288/2, 3557066/2, 1836827, 723584]
#FLUX_ADU_STAR = [9041644.0, 1778532.9, 1836827, 723584]
INFO = ['2019-07-09', '2016-04-05', '2016-06-03', '2018-06-01']

In [58]:
## Params: flux

for i in range(len(FLUX_ADU_OBJ)):
    TRANSMISSION_STAR = TRANSMISSION_STAR_S[i]
    DIT_OBJECT, DIT_STAR = DIT_OBJECT_S[i], DIT_STAR_S[i]
    
    flux_adu_obj = FLUX_ADU_OBJ[i] # total flux of the disk after multplying by the scaling parameter
    flux_adu_star = FLUX_ADU_STAR[i] # within a box region of size = 25 pixels, left+right images
    
    print('\n== INFO epoch', INFO[i], '==')
    #print('(results *not* corrected from DITs and transmission)')
    #print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
    #print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))

    print('(results corrected from DITs and transmission)')
    print('The flux of the star is {:.4e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
    print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

    flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
            FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux of the disk is {:.4e} Jy/arcsec^2.'.format(flux_obj_final_unit))


== INFO epoch 2019-07-09 ==
(results corrected from DITs and transmission)
The flux of the star is 1.9377e+07 ADU/s.
The flux of the disk is 209.66 ADU/s
The flux of the disk is 7.0300e-02 Jy/arcsec^2.

== INFO epoch 2016-04-05 ==
(results corrected from DITs and transmission)
The flux of the star is 3.0478e+07 ADU/s.
The flux of the disk is 341.09 ADU/s
The flux of the disk is 7.2714e-02 Jy/arcsec^2.

== INFO epoch 2016-06-03 ==
(results corrected from DITs and transmission)
The flux of the star is 3.1477e+07 ADU/s.
The flux of the disk is 278.12 ADU/s
The flux of the disk is 5.7408e-02 Jy/arcsec^2.

== INFO epoch 2018-06-01 ==
(results corrected from DITs and transmission)
The flux of the star is 6.8619e+06 ADU/s.
The flux of the disk is 283.95 ADU/s
The flux of the disk is 2.6886e-01 Jy/arcsec^2.



== INFO epoch 2019-07-09 ==
(results corrected from DITs and transmission)
The flux of the star is 1.9377e+07 ADU/s.
The flux of the disk is 209.66 ADU/s
The flux of the disk is 7.0300e-02 Jy/arcsec^2.

== INFO epoch 2016-04-05 ==
(results corrected from DITs and transmission)
The flux of the star is 3.0478e+07 ADU/s.
The flux of the disk is 341.09 ADU/s
The flux of the disk is 7.2714e-02 Jy/arcsec^2.

== INFO epoch 2016-06-03 ==
(results corrected from DITs and transmission)
The flux of the star is 3.1477e+07 ADU/s.
The flux of the disk is 278.12 ADU/s
The flux of the disk is 5.7408e-02 Jy/arcsec^2.

== INFO epoch 2018-06-01 ==
(results corrected from DITs and transmission)
The flux of the star is 6.8619e+06 ADU/s.
The flux of the disk is 283.95 ADU/s
The flux of the disk is 2.6886e-01 Jy/arcsec^2.


### Results from joint-fitting total intensity in both BBH and H23 bands 

##### for the total intensity-part: used 5 modes, 1 channel

In [80]:
DIT_OBJECT_S = [32, 4.001856, 4.001856]
DIT_STAR_S   = [32, 4.001856, 4.001856]

TRANSMISSION_STAR_S = [0.014581900, 0.014581900, 0.014581900]

FLUX_ADU_OBJ  = [8819, 1231, 1691]
FLUX_ADU_STAR = [9028321/2, 1768472/2, 925421]
INFO = ['2019-07-09', '2016-04-05', '2016-06-03']

In [79]:
## Params: flux
for i in range(len(FLUX_ADU_OBJ)):
    TRANSMISSION_STAR = TRANSMISSION_STAR_S[i]
    DIT_OBJECT, DIT_STAR = DIT_OBJECT_S[i], DIT_STAR_S[i]
    
    flux_adu_obj = FLUX_ADU_OBJ[i] # total flux of the disk after multplying by the scaling parameter
    flux_adu_star = FLUX_ADU_STAR[i] # within a box region of size = 25 pixels, left+right images
    
    print('\n== INFO epoch', INFO[i], '==')
    #print('(results *not* corrected from DITs and transmission)')
    #print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
    #print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))

    print('(results corrected from DITs and transmission)')
    print('The flux of the star is {:.1e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
    print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

    flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
            FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux of the disk is {:.2e} Jy/arcsec^2.'.format(flux_obj_final_unit))


== INFO epoch 2019-07-09 ==
(results corrected from DITs and transmission)
The flux of the star is 9.7e+06 ADU/s.
The flux of the disk is 275.59 ADU/s
The flux of the disk is 1.85e-01 Jy/arcsec^2.

== INFO epoch 2016-04-05 ==
(results corrected from DITs and transmission)
The flux of the star is 1.5e+07 ADU/s.
The flux of the disk is 307.61 ADU/s
The flux of the disk is 1.32e-01 Jy/arcsec^2.

== INFO epoch 2016-06-03 ==
(results corrected from DITs and transmission)
The flux of the star is 1.6e+07 ADU/s.
The flux of the disk is 422.55 ADU/s
The flux of the disk is 1.73e-01 Jy/arcsec^2.


##### for the total intensity-part: used 5 modes, both channels

In [82]:
DIT_OBJECT_S = [32, 4.001856, 4.001856]
DIT_STAR_S   = [32, 4.001856, 4.001856]

TRANSMISSION_STAR_S = [0.014581900, 0.014581900, 0.014581900]

FLUX_ADU_OBJ  = [18956, 3294, 3043]
FLUX_ADU_STAR = [18083288/2, 3557066/2, 1836827]
INFO = ['2019-07-09', '2016-04-05', '2016-06-03']

In [83]:
## Params: flux
for i in range(len(FLUX_ADU_OBJ)):
    TRANSMISSION_STAR = TRANSMISSION_STAR_S[i]
    DIT_OBJECT, DIT_STAR = DIT_OBJECT_S[i], DIT_STAR_S[i]
    
    flux_adu_obj = FLUX_ADU_OBJ[i] # total flux of the disk after multplying by the scaling parameter
    flux_adu_star = FLUX_ADU_STAR[i] # within a box region of size = 25 pixels, left+right images
    
    print('\n== INFO epoch', INFO[i], '==')
    #print('(results *not* corrected from DITs and transmission)')
    #print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
    #print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))

    print('(results corrected from DITs and transmission)')
    print('The flux of the star is {:.1e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
    print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

    flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
            FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux of the disk is {:.2e} Jy/arcsec^2.'.format(flux_obj_final_unit))


== INFO epoch 2019-07-09 ==
(results corrected from DITs and transmission)
The flux of the star is 1.9e+07 ADU/s.
The flux of the disk is 592.38 ADU/s
The flux of the disk is 1.99e-01 Jy/arcsec^2.

== INFO epoch 2016-04-05 ==
(results corrected from DITs and transmission)
The flux of the star is 3.0e+07 ADU/s.
The flux of the disk is 823.12 ADU/s
The flux of the disk is 1.75e-01 Jy/arcsec^2.

== INFO epoch 2016-06-03 ==
(results corrected from DITs and transmission)
The flux of the star is 3.1e+07 ADU/s.
The flux of the disk is 760.40 ADU/s
The flux of the disk is 1.57e-01 Jy/arcsec^2.


##### for the total intensity-part: used 10 modes, 1 channel

In [84]:
DIT_OBJECT_S = [32, 4.001856, 4.001856]
DIT_STAR_S   = [32, 4.001856, 4.001856]

TRANSMISSION_STAR_S = [0.014581900, 0.014581900, 0.014581900]

FLUX_ADU_OBJ  = [5000, 1176, 1071]
FLUX_ADU_STAR = [9028321/2, 1768472/2, 925421, 723584]
INFO = ['2019-07-09', '2016-04-05', '2016-06-03']

In [85]:
## Params: flux
for i in range(len(FLUX_ADU_OBJ)):
    TRANSMISSION_STAR = TRANSMISSION_STAR_S[i]
    DIT_OBJECT, DIT_STAR = DIT_OBJECT_S[i], DIT_STAR_S[i]
    
    flux_adu_obj = FLUX_ADU_OBJ[i] # total flux of the disk after multplying by the scaling parameter
    flux_adu_star = FLUX_ADU_STAR[i] # within a box region of size = 25 pixels, left+right images
    
    print('\n== INFO epoch', INFO[i], '==')
    #print('(results *not* corrected from DITs and transmission)')
    #print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
    #print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))

    print('(results corrected from DITs and transmission)')
    print('The flux of the star is {:.1e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
    print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

    flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
            FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux of the disk is {:.2e} Jy/arcsec^2.'.format(flux_obj_final_unit))


== INFO epoch 2019-07-09 ==
(results corrected from DITs and transmission)
The flux of the star is 9.7e+06 ADU/s.
The flux of the disk is 156.25 ADU/s
The flux of the disk is 1.05e-01 Jy/arcsec^2.

== INFO epoch 2016-04-05 ==
(results corrected from DITs and transmission)
The flux of the star is 1.5e+07 ADU/s.
The flux of the disk is 293.86 ADU/s
The flux of the disk is 1.26e-01 Jy/arcsec^2.

== INFO epoch 2016-06-03 ==
(results corrected from DITs and transmission)
The flux of the star is 1.6e+07 ADU/s.
The flux of the disk is 267.63 ADU/s
The flux of the disk is 1.10e-01 Jy/arcsec^2.


##### for the total intensity-part: used 10 modes, both channels

In [86]:
DIT_OBJECT_S = [32, 4.001856, 4.001856]
DIT_STAR_S   = [32, 4.001856, 4.001856]

TRANSMISSION_STAR_S = [0.014581900, 0.014581900, 0.014581900]

FLUX_ADU_OBJ  = [8510, 1728, 1201]
FLUX_ADU_STAR = [18083288/2, 3557066/2, 1836827, 723584]
INFO = ['2019-07-09', '2016-04-05', '2016-06-03']

In [87]:
## Params: flux
for i in range(len(FLUX_ADU_OBJ)):
    TRANSMISSION_STAR = TRANSMISSION_STAR_S[i]
    DIT_OBJECT, DIT_STAR = DIT_OBJECT_S[i], DIT_STAR_S[i]
    
    flux_adu_obj = FLUX_ADU_OBJ[i] # total flux of the disk after multplying by the scaling parameter
    flux_adu_star = FLUX_ADU_STAR[i] # within a box region of size = 25 pixels, left+right images
    
    print('\n== INFO epoch', INFO[i], '==')
    #print('(results *not* corrected from DITs and transmission)')
    #print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
    #print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))

    print('(results corrected from DITs and transmission)')
    print('The flux of the star is {:.1e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
    print('The flux of the disk is {:.2f} ADU/s'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

    flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
            FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

    print('The flux of the disk is {:.2e} Jy/arcsec^2.'.format(flux_obj_final_unit))


== INFO epoch 2019-07-09 ==
(results corrected from DITs and transmission)
The flux of the star is 1.9e+07 ADU/s.
The flux of the disk is 265.94 ADU/s
The flux of the disk is 8.92e-02 Jy/arcsec^2.

== INFO epoch 2016-04-05 ==
(results corrected from DITs and transmission)
The flux of the star is 3.0e+07 ADU/s.
The flux of the disk is 431.80 ADU/s
The flux of the disk is 9.21e-02 Jy/arcsec^2.

== INFO epoch 2016-06-03 ==
(results corrected from DITs and transmission)
The flux of the star is 3.1e+07 ADU/s.
The flux of the disk is 300.11 ADU/s
The flux of the disk is 6.19e-02 Jy/arcsec^2.


# 2. From processed data

From polarimetry maybe relevant but be careful for total imaging: self-subtraction effect !! so underestimation of the flux on the processed image.


# Polarimetry

**Goal:** From Qphi image, we have values in ADU We want to convert this ADU value in a flux value.


$$\text{QPHI (ADU)} \; \rightarrow\; \text{QPHI (Jy)}\; =\; \frac{\text{QPHI (ADU)}}{\text{STAR (ADU)}} \times \frac{\text{DIT}_\star}{\text{DIT}_\text{QPHI}} \times \frac{\text{TRANSMISSION}_\star}{\text{TRANSMISSION}_\text{QPHI}}  \times \frac{1}{\text{PLATESCALE (arcsec/pixel)}^2} \times \text{STAR (Jy)} $$  

Values of DIT_STAR, DIT_OBJECT, TRANSMISSION_STAR, TRANSMISSION_OBJECT are taken from the header of the cubes.

In [48]:
## Params: obs
DIT_STAR, DIT_OBJECT = 0.837464, 64 # s taken from the header of the images
TRANSMISSION_STAR, TRANSMISSION_OBJECT = 0.12591487665785145, 1 # corresponds to ND1.0 and OPEN; taken from the header of the images
PLATESCALE =  12.25e-3 # arcsec/pix 
FLUX_REF_STAR = 0.975 # Jy from Simbad in the filter Johnson H

In [49]:
#DIT_STAR/DIT_OBJECT

In [50]:
#TRANSMISSION_STAR/TRANSMISSION_OBJECT

Note: for the polarimetric image, IRDAP considers both left and right cubes. Thus, when we consider the stellar flux, we sum the values of the left and right images within a circular of a given radius.

By default, a radius of 120 pixels is used in the IRDAP processing. However, as we would like to compare polarised fraction to total intensity fraction, and that in total intensity the PSF file ("median_unsat.fits") is smaller (64x64 pixels^2), we use a radius of 30 pixels to derive the stellar flux in polarimetry and in total intensity. 



In [31]:
## Params: flux
flux_adu_obj = 25 # value of a pixel in a bright location of the disk in the QPHI IRDAP-processed image.
flux_adu_star = (194015+195473) # within circular region of r = 30 pixels, left+right images
#flux_adu_star = 216706 # within circular region of r = 120 pixels. Default is r = 120 pixels.


print('(results *not* corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
print('The flux of the disk is {:.2f} ADU.'.format(flux_adu_obj))


print('\n(results corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
print('The flux of the disk is {:.2f} ADU/s(!)'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

(results *not* corrected from DITs and transmission)
The flux of the star is 3.9e+05 ADU.
The flux of the disk is 25.00 ADU.

(results corrected from DITs and transmission)
The flux of the star is 3.7e+06 ADU/s.
The flux of the disk is 0.39 ADU/s(!)


In [32]:
flux_obj_final_unit =  ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.1e} Jy/arcsec^2.'.format(flux_obj_final_unit))

Star flux = 3.694e+06 ADU/s
The flux of the disk is 6.9e-04 Jy/arcsec^2.


# Total intensity

For the star flux, we have two PSF acquired before and after the observation and for the left and right part of the detector.
We sum the flux of the left and rights images, and take the mean from before and after the observation.

Values of DIT_STAR, DIT_OBJECT, TRANSMISSION_STAR, TRANSMISSION_OBJECT are taken from the header of the cubes.

In [33]:
# Params
DIT_STAR, DIT_OBJECT = 8, 32 # s
TRANSMISSION_STAR, TRANSMISSION_OBJECT = 0.014581900, 1 # ND2.0 and OPEN are used. Corresponding value for ND2.0
# taken from the ESO website https://www.eso.org/sci/facilities/paranal/instruments/sphere/inst/filters.html
PLATESCALE =  12.25e-3 # arcsec/pix 
FLUX_REF_STAR = 0.975  # Jy from Simbad in the filter Johnson H

#### Consider only one side of the detector (indexed as 0; first channel)

For the star flux, we have two PSF acquired before and after the observation and for the left and right part of the detector.
We consider the psf only on one part of the cube (indexed as 0; first channel), and take the mean from before and after the observation.

In [37]:
# STAR
# within circular region of r = 30 pixels centered in (33,33)
flux_adu_star = np.array( [[5107827, 5137693], [4738552, 4768522]]) # within box region of size = 30 pixels centered in (33,33)
flux_adu_star = np.nanmean( flux_adu_star[:,0] )


# The science reduced image is given in contrast; multiply by a normalization factor to have it back in ADU
normalization_factor = np.array([7774262.5, 7792016]) 
normalization_factor = np.mean(normalization_factor)

flux_adu_obj = 7.62037e-08 * normalization_factor

print('(results *not* corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))
print('The flux of the disk is {:.2f} ADU(!)'.format(flux_adu_obj))


print('\n(results corrected from DITs and transmission)')
print('The flux of the star is {:.1e} ADU/s.'.format(flux_adu_star/(DIT_STAR*TRANSMISSION_STAR)))
print('The flux of the disk is {:.2f} ADU/s(!)'.format(flux_adu_obj/(DIT_OBJECT*TRANSMISSION_OBJECT)))

(results *not* corrected from DITs and transmission)
The flux of the star is 4.9e+06 ADU.
The flux of the disk is 0.59 ADU(!)

(results corrected from DITs and transmission)
The flux of the star is 4.2e+07 ADU/s.
The flux of the disk is 0.02 ADU/s(!)


Weird that the flux value of the disk in total intensity is lower than the flux value derived from polarimetry. From a factor 20 (!)  
Célia's guess: The flux value derived for the disk suffers from self-subtraction.

In [23]:
flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.1e} Jy/arcsec^2.'.format(flux_obj_final_unit))

Star flux = 4.220e+07 ADU/s
The flux of the disk is 2.9e-06 Jy/arcsec^2.


#### Sum over the two wavelengths

For the star flux, we have two PSF acquired before and after the observation and for the left and right part of the detector.
We sum the flux of the left and rights images, and take the mean from before and after the observation.

In [152]:
# STAR
# within circular region of r = 30 pixels centered in (33,33)
flux_adu_star = np.array( [[5107827, 5137693], [4738552, 4768522]]) # within box region of size = 30 pixels centered in (33,33)
flux_adu_star = np.nanmean( np.nansum(flux_adu_star, axis=1) )

print('The flux of the star is {:.1e} ADU.'.format(flux_adu_star))

# The science reduced image is given in contrast; multiply by a normalization factor to have it back in ADU
normalization_factor = np.array([7774262.5, 7792016]) 
normalization_factor = np.mean(normalization_factor)

flux_adu_obj = 7.62037e-08 * normalization_factor * 2 # multiply by 2 because consider two bands so twice photons more
print('The flux of the disk is {:.2f} ADU (!)'.format(flux_adu_obj))

The flux of the star is 9.9e+06 ADU.
The flux of the disk is 1.19 ADU (!)


Weird that the value is so low compared to polarimetry?!

In [153]:
flux_obj_final_unit = ADU2FLUX(flux_adu_obj, flux_adu_star,
        FLUX_REF_STAR, DIT_OBJECT, DIT_STAR, TRANSMISSION_OBJECT, TRANSMISSION_STAR, PLATESCALE)

print('The flux of the disk is {:.1e} Jy/arcsec^2.'.format(flux_obj_final_unit))

Star flux = 8.466e+07 ADU/s
The flux of the disk is 2.8e-06 Jy/arcsec^2.


In [154]:
flux_adu_star

9876297.0

# Test

Comparison PCA processed images, 10 modes, pixel (296,308) ds9 convention with normalization (by the PSF, contrast values) or without normalization (still ADU)

In [42]:
flux_1 = 1.1959697
flux_norm = 1.1825136e-7 
flux_1/flux_norm

10113792.348772988

In [46]:
flux_1/normalization_factor * 1/flux_norm

1.2994489786075698

In [ ]:
flux_adu_star/(flux_1/flux_norm)

In [45]:
0.092981704/2.3942583e-8

3883528.5232173987